# Procesamiento Geoespacial Optimizado

Este notebook demuestra técnicas optimizadas para procesar grandes conjuntos de datos geoespaciales sin que el kernel de Jupyter se bloquee.

## 1. Configuración inicial y carga de dependencias

In [ ]:
# Importaciones necesarias
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import logging
from datetime import datetime

# Asegurarse de que el directorio principal está en el path
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Importar utilidades propias
from utils.memory_utils import (
    print_memory_usage, limit_memory_usage, set_memory_limit, 
    load_data_with_chunks, process_by_chunks, mask_and_align_coordinates_safe
)
from utils.custom_logger import setup_logger

# Configurar logging
log_file = os.path.join(project_dir, 'logs', f'log-{datetime.now().strftime("%Y-%m-%d")}.log')
logger = setup_logger(log_file)

# Configurar matplotlib para visualización
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Configurar límites de memoria para prevenir bloqueos

Limitar la memoria que puede usar el kernel ayuda a prevenir que se bloquee.

In [ ]:
# Establecer límite al 80% de la memoria disponible
set_memory_limit(80)

# Ver uso actual de memoria
print_memory_usage()

## 3. Cargar datos con chunkings

En lugar de cargar todo el conjunto de datos en memoria, cargamos porciones manejables.

In [ ]:
# Rutas a los datasets (ajustar según tu configuración)
CHIRPS_MONTHLY_PATH = "/Users/riperez/Conda/anaconda3/doc/precipitation/output/boyaca_region_monthly_sum.nc"
DEM_PATH = "/Users/riperez/Conda/anaconda3/doc/precipitation/output/dem_boyaca_90.nc"

# Cargar datasets con chunks
dem_ds = load_data_with_chunks(DEM_PATH, {'latitude': 500, 'longitude': 500})
chirps_ds = load_data_with_chunks(CHIRPS_MONTHLY_PATH, {'latitude': 500, 'longitude': 500, 'time': 10})

# Información básica sobre los datasets
print("\nInformación del dataset DEM:")
print(dem_ds.info())

print("\nInformación del dataset CHIRPS:")
print(chirps_ds.info())

## 4. Procesamiento por chunks

En lugar de procesar todo el conjunto de datos a la vez, lo dividimos en trozos más pequeños y los procesamos uno por uno.

In [ ]:
# Dividir el dataset DEM en chunks más pequeños
dem_chunks = process_by_chunks(dem_ds, chunk_size=400)

# Ver cuántos chunks tenemos
print(f"Número de chunks DEM: {len(dem_chunks)}")
print(f"Tamaño del primer chunk: {dem_chunks[0].dims}")

# Mostrar memoria después de chunking
print_memory_usage()

## 5. Alineación de coordenadas segura

Esta es una versión optimizada de mask_and_align_coordinates que evita bloqueos del kernel al procesar por lotes.

In [ ]:
# Alinear coordenadas entre datasets DEM y CHIRPS
dem_aligned, chirps_aligned = mask_and_align_coordinates_safe(dem_ds, chirps_ds, batch_size=300)

# Ver información sobre los datasets alineados
print("\nDimensiones DEM alineado:")
print(dem_aligned.dims)

print("\nDimensiones CHIRPS alineado:")
print(chirps_aligned.dims)

## 6. Análisis estadístico seguro con control de memoria

In [ ]:
def calculate_stats_safely(dataset, variable_name):
    """Calcula estadísticas básicas de manera segura para la memoria"""
    # Ver uso de memoria antes de empezar
    print_memory_usage()
    
    stats = {}
    
    # Intentar calcular estadísticas básicas
    try:
        stats['mean'] = float(dataset[variable_name].mean().values)
        # Liberar memoria después de cada operación
        gc.collect()
        
        stats['std'] = float(dataset[variable_name].std().values)
        gc.collect()
        
        stats['min'] = float(dataset[variable_name].min().values)
        gc.collect()
        
        stats['max'] = float(dataset[variable_name].max().values)
        gc.collect()
        
        # Para percentiles, usamos numpy con .values para controlar mejor la memoria
        # Convertimos a valores numpy, aplicamos función y liberamos memoria rápido
        values = dataset[variable_name].values.flatten()
        mask = ~np.isnan(values)
        valid_values = values[mask]
        stats['percentile_25'] = np.percentile(valid_values, 25)
        stats['median'] = np.percentile(valid_values, 50)
        stats['percentile_75'] = np.percentile(valid_values, 75)
        
        # Liberar memoria explícitamente
        del values, mask, valid_values
        gc.collect()
        
    except Exception as e:
        logging.error(f"Error calculando estadísticas para {variable_name}: {e}")
    
    # Ver uso de memoria después de terminar
    print_memory_usage()
    
    return stats

# Calcular estadísticas para DEM
if 'DEM' in dem_aligned:
    dem_stats = calculate_stats_safely(dem_aligned, 'DEM')
    print("Estadísticas DEM:")
    for key, value in dem_stats.items():
        print(f"{key}: {value}")

# Calcular estadísticas para precipitación
if 'total_precipitation' in chirps_aligned:
    precip_stats = calculate_stats_safely(chirps_aligned, 'total_precipitation')
    print("\nEstadísticas de Precipitación:")
    for key, value in precip_stats.items():
        print(f"{key}: {value}")

## 7. Visualización segura de datos

Visualizar datasets grandes puede consumir mucha memoria. Aquí demostramos cómo hacerlo de forma segura.

In [ ]:
def plot_safely(dataset, variable, time_index=0):
    """Visualiza datos geoespaciales de forma segura"""
    try:
        # Si tenemos dimensión de tiempo, seleccionar un slice
        if 'time' in dataset.dims:
            # Tomar solo un punto de tiempo para visualizar
            data_to_plot = dataset[variable].isel(time=time_index)
            title = f"{variable} - Tiempo: {dataset.time.values[time_index]}"
        else:
            data_to_plot = dataset[variable]
            title = f"{variable}"
        
        # Reducir resolución para visualización si el dataset es muy grande
        if data_to_plot.size > 1_000_000:  # 1 millón de puntos
            logging.info("Dataset muy grande, reduciendo resolución para visualización")
            # Tomar 1 de cada N puntos
            step = max(1, int(data_to_plot.size / 500_000))
            if 'latitude' in data_to_plot.dims and 'longitude' in data_to_plot.dims:
                data_to_plot = data_to_plot.isel(
                    latitude=slice(None, None, step),
                    longitude=slice(None, None, step)
                )
        
        # Crear figura
        plt.figure(figsize=(10, 8))
        
        # Graficar
        data_to_plot.plot(cmap='viridis')
        plt.title(title)
        plt.colorbar(label=variable)
        
        # Mostrar
        plt.tight_layout()
        plt.show()
        
        # Liberar memoria
        plt.close()
        del data_to_plot
        gc.collect()
        
    except Exception as e:
        logging.error(f"Error al visualizar {variable}: {e}")

# Visualizar DEM
if 'DEM' in dem_aligned:
    plot_safely(dem_aligned, 'DEM')

# Visualizar precipitación para diferentes puntos de tiempo
if 'total_precipitation' in chirps_aligned and 'time' in chirps_aligned.dims:
    # Visualizar primero, medio y último punto temporal
    time_points = [0, len(chirps_aligned.time)//2, -1]
    for t in time_points:
        plot_safely(chirps_aligned, 'total_precipitation', time_index=t)

## 8. Cálculo de correlaciones por bloques

Calcular correlaciones entre datos de precipitación y elevación puede consumir mucha memoria cuando se hace todo a la vez. Aquí lo hacemos por bloques.

In [ ]:
def calculate_correlation_by_blocks(dem_data, precip_data, block_size=200):
    """Calcula correlación entre DEM y precipitación por bloques"""
    # Verificar que tenemos las dimensiones espaciales correctas
    dem_lat_name = 'latitude' if 'latitude' in dem_data.dims else 'lat'
    dem_lon_name = 'longitude' if 'longitude' in dem_data.dims else 'lon'
    
    precip_lat_name = 'latitude' if 'latitude' in precip_data.dims else 'lat'
    precip_lon_name = 'longitude' if 'longitude' in precip_data.dims else 'lon'
    
    # Colapsar la dimensión temporal para precipitación (promedio)
    if 'time' in precip_data.dims:
        precip_mean = precip_data['total_precipitation'].mean(dim='time')
    else:
        precip_mean = precip_data['total_precipitation']
    
    # Inicializar array para correlaciones
    corr_data = np.zeros((len(precip_data[precip_lat_name]), len(precip_data[precip_lon_name])))
    corr_data[:] = np.nan  # Inicializar con NaN
    
    # Obtener arrays numpy para cálculos más eficientes
    dem_values = dem_data['DEM'].values
    precip_values = precip_mean.values
    
    # Procesar por bloques para evitar problemas de memoria
    for i in range(0, dem_values.shape[0], block_size):
        i_end = min(i + block_size, dem_values.shape[0])
        
        for j in range(0, dem_values.shape[1], block_size):
            j_end = min(j + block_size, dem_values.shape[1])
            
            # Extraer bloque
            dem_block = dem_values[i:i_end, j:j_end]
            precip_block = precip_values[i:i_end, j:j_end]
            
            # Calcular correlación para este bloque si hay suficientes datos válidos
            mask = ~(np.isnan(dem_block) | np.isnan(precip_block))
            if np.sum(mask) > 10:  # Asegurar que tenemos suficientes puntos
                dem_flat = dem_block[mask]
                precip_flat = precip_block[mask]
                
                # Calcular coeficiente de correlación
                if len(dem_flat) > 0:
                    corr = np.corrcoef(dem_flat, precip_flat)[0, 1]
                    # Almacenar solo en el centro del bloque para simplificar
                    center_i = i + (i_end - i) // 2
                    center_j = j + (j_end - j) // 2
                    if center_i < corr_data.shape[0] and center_j < corr_data.shape[1]:
                        corr_data[center_i, center_j] = corr
            
            # Liberar memoria
            if (j_end % (block_size * 5)) == 0:
                gc.collect()
                print_memory_usage()
    
    # Crear DataArray con las correlaciones
    corr_da = xr.DataArray(
        corr_data,
        dims=[precip_lat_name, precip_lon_name],
        coords={
            precip_lat_name: precip_data[precip_lat_name],
            precip_lon_name: precip_data[precip_lon_name]
        },
        name="dem_precip_correlation"
    )
    
    return corr_da

# Calcular correlaciones si tenemos datos alineados
if 'DEM' in dem_aligned and 'total_precipitation' in chirps_aligned:
    print("Calculando correlación entre elevación y precipitación...")
    correlation = calculate_correlation_by_blocks(dem_aligned, chirps_aligned, block_size=150)
    
    # Visualizar correlación
    plt.figure(figsize=(12, 8))
    correlation.plot(cmap='RdBu_r', vmin=-1, vmax=1)
    plt.title('Correlación entre Elevación y Precipitación')
    plt.colorbar(label='Coeficiente de Correlación')
    plt.tight_layout()
    plt.show()

## 9. Guardar resultados con control de memoria

Finalmente, guardamos resultados pero asegurándonos de no sobrecargar la memoria.

In [ ]:
def save_dataset_safely(dataset, filepath, chunks=None):
    """Guarda un dataset en archivo netCDF de forma segura"""
    logging.info(f"Guardando dataset en {filepath}...")
    
    try:
        # Si el dataset es muy grande, asegurar chunking para escritura
        if chunks is not None and dataset.nbytes > 1e9:  # 1 GB
            dataset = dataset.chunk(chunks)
        
        # Crear directorio si no existe
        os.makedirs(os.path.dirname(os.path.abspath(filepath)), exist_ok=True)
        
        # Guardar archivo
        dataset.to_netcdf(filepath)
        logging.info(f"Dataset guardado exitosamente en {filepath}")
        return True
    except Exception as e:
        logging.error(f"Error guardando dataset: {e}")
        return False

# Guardar resultados con control de chunks
output_dir = os.path.join(project_dir, 'data', 'transformation', 'build')
os.makedirs(output_dir, exist_ok=True)

# Guardar dataset de correlación
if 'correlation' in locals():
    correlation_ds = xr.Dataset({'dem_precip_correlation': correlation})
    save_dataset_safely(
        correlation_ds, 
        os.path.join(output_dir, 'dem_precip_correlation_optimized.nc'),
        chunks={'latitude': 200, 'longitude': 200}
    )

## 10. Limpiar recursos y liberar memoria

Es importante limpiar recursos al final del notebook para evitar bloqueos al ejecutar otras celdas o notebooks.

In [ ]:
# Cerrar todos los datasets abiertos
for var_name in ['dem_ds', 'chirps_ds', 'dem_aligned', 'chirps_aligned']:
    if var_name in locals():
        try:
            locals()[var_name].close()
            logging.info(f"Dataset {var_name} cerrado exitosamente")
        except:
            pass

# Cerrar figuras de matplotlib
plt.close('all')

# Forzar recolección de basura
gc.collect()

# Verificar uso final de memoria
mem_usage = print_memory_usage()
print(f"\nProcesamiento finalizado. Uso final de memoria: {mem_usage:.2f} MB")